In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Hyperparameters Results
# not too high to avoid overfitting (lower or equals to 12 ?)
max_depthFound=14
min_samples_leafFound=7
min_samples_splitFound=24
criterionFound='gini'

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

issue overfitting (train good results, test poor results)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = DecisionTreeClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('06-sklearn.tree','DecisionTree','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('06-sklearn.tree','DecisionTree','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
#fd.show_precision_recall_curve(modelClf,x_train,y_train,x_test,y_test)
fd.show_prediction_graph(modelClf, x_test,y_test)



# Scaling choice
with decision tree the scaling has no influence

# Hyperparameters tuning

In [ ]:
%%script false
# to avoid overfitting 12
# found = 14

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []
# too low max_depth -> no result
for max_depth in np.arange(1,23,1):
    print('max_depth',max_depth)
    # Instantiate model and k-fold
    modelClf = DecisionTreeClassifier(random_state=42, max_depth=max_depth,
        criterion=criterionFound,min_samples_leaf=min_samples_leafFound, min_samples_split=min_samples_splitFound)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1", test_f1)
    print("dif", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(max_depth)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

In [ ]:
%%script false

from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(6,15),
    'min_samples_leaf': randint(5,25),
    'min_samples_split': randint(20,40),
}

#{'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 10, 'min_samples_split': 30}
#0.14124665444787682
#score   0.23259493670886075
#scoref1 0.23259493670886075


res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

In [ ]:
%%script false

modelClf = DecisionTreeClassifier(random_state=42)
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': [9,10,11,12,13,14],
    'min_samples_leaf': [7,8,9,10,11,12],
    'min_samples_split': [24,26,28,30,32]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
#{'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 7, 'min_samples_split': 22}
#0.16394089663249284
#scoref1 0.2450592885375494

#{'criterion': 'gini', 'max_depth': 14, 'min_samples_leaf': 7, 'min_samples_split': 24}
#0.16057730710563736
#scoref1 0.24784313725490195


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('06-sklearn.tree','DecisionTree','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('06-sklearn.tree','DecisionTree','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD06A-DecisionTreeClassifierProbaHistogram' if saveImage else None)
#fd.show_precision_recall_curve(modelClf,x_train,y_train,x_test,y_test)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
# Plot the decision tree
plt.figure(figsize=(20, 10))
plot_tree(modelClf, filled=True, feature_names=dfTrx.columns, class_names=['Geniune', 'Fraudulent'])
plt.show()


In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('06-sklearn.tree','DecisionTreeClassifier','2-After tuning', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

